In [ ]:
!pip install -U transformers==4.43.3
#!pip install -U git+https://github.com/huggingface/transformers
!pip install -U accelerate==0.33.0
!pip install -U torch==2.1.1
!pip install datasets==2.20.0
#!pip install -U trl
!pip install git+https://github.com/huggingface/trl.git
!pip install -q -U bitsandbytes==0.43.3 einops==0.8.0 wandb
!pip install -U huggingface-hub==0.24.5
# peft es una libreria para calcular la eficiencia de la aceleración de un modelo
!pip install -U tokenizers==0.19.1
!pip install -U peft==0.12.0
!pip install load_dotenv

In [ ]:
# Importar las bibliotecas necesarias
# Load model directly
from transformers import (
    AutoModelForCausalLM, 
    AutoTokenizer, 
)
import torch, load_dotenv, os
import ipywidgets as widgets
from IPython.display import display, HTML
from huggingface_hub import login # huggingface_hub es de hugingface

In [ ]:
if load_dotenv.load_dotenv('../env.env'):
    access_token = os.getenv('acesstoken' )
    login(token=access_token, add_to_git_credential=True)
    print("authenticated")

In [ ]:
# Load model directly
model = AutoModelForCausalLM.from_pretrained(
    "BrunoGR/Just_HEAR_Me",
    torch_dtype=torch.float16,
    #quantization_config=bnb_config,
    load_in_8bit=True,
    device_map="cuda"
)

In [ ]:
tokenizer = AutoTokenizer.from_pretrained("BrunoGR/Just_HEAR_Me")

In [ ]:
# Configurar la interfaz de usuario
input_box = widgets.Text(
    placeholder='Escribe tu mensaje aquí...',
    description='Mensaje:',
    disabled=False
)

def respond_to_message(message:str,prev:str="",temperature=0.7, top_k=50, top_p=0.9, num_beams=5):
    # Tokenizar el mensaje de entrada
    prompt= '''### instruction: 
    You are an emotional assistant, respond empathetically in Spanish to each of the messages. If the user seems sad or upset, you should offer words of encouragement. If the user seems happy or excited, the assistant should share that joy and respond enthusiastically. In all cases, the assistant should maintain a respectful tone, if possible, encouraging you to talk more about it. Don't say hello, unless necessary. Use the username and pronoun to respond in a personalized way. 
    ### input:'''
    message= prompt+prev+"User:\n"+ message+"\nemotional assistant(response in spanish):"
    input_ids = tokenizer.encode(message + tokenizer.eos_token, return_tensors='pt')
    input_ids = input_ids.to('cuda')

    # Generar la respuesta del modelo
    chat_history_ids = model.generate(input_ids, 
                                      max_length=4000,
                                      num_beams=num_beams,
                                      temperature=temperature,
                                      top_k=top_k,
                                      top_p=top_p,
                                     early_stopping=True)

    # Decodificar la respuesta
    response = tokenizer.decode(chat_history_ids[:, input_ids.shape[-1]:][0], skip_special_tokens=True)

    return response

def on_message_submit(user_message:str, prev:str ="",temperature=0.7, top_k=50, top_p=0.9, num_beams=5):
    
    response = respond_to_message(user_message, prev, temperature, top_k, top_p, num_beams)
    conversation= f"{prev}User: {user_message}\nemotional assistant: {response}\n"
    return conversation





In [ ]:
prev=''

In [ ]:
prev_s='''User: Hola!
emotional assistant: 
Hola, ¿cómo estás hoy? ¿Hay algo en lo que pueda ayudarte?
User: Estoy muy bien y tu?
emotional assistant: 
Me alegra saber que estás bien. ¿Hay algo en lo que pueda ayudarte? Estoy aquí para escucharte.
User: Pues hoy tuve un buen dia, quieres escucharlo?
emotional assistant: 
Me alegra mucho que hayas tenido un buen día. Cuéntame más sobre lo que sucedió.
User: Pues mira, te cuento que hoy en mi trabajo me dijeron que las cosas se estan poniendo interesantes, aun mas con el proyecto que estoy desarrollando, porque a unos proveedores les intereso, hoy agendaron una cita.
emotional assistant: 
¡Eso suena emocionante! Es genial cuando las cosas se ponen interesantes en el trabajo. ¿Cómo te sientes al respecto?
User: Pues bastante emocionado, pero nervioso a la vez, no se como decirlo, pero tengo miedo a fallar y que cancelen todo el proyecto.
emotional assistant: 
Comprendo tu nerviosismo, pero recuerda que todos cometemos errores y lo importante es aprender de ellos. Confía en tus habilidades y en el proyecto que estás desarrollando. Estoy aquí para apoyarte en lo que necesites.
'''

In [ ]:
prev_a='''User: Este fin de semana que estuve con mis amigos, me di cuenta de algo muy incomodo, pero si te soy sincero no se como actuar al respecto. Me di cuenta que mi amigo Alex, le esta hablando a mi ex pareja, yo creo que el no se dio cuenta, perocuando se me acerco para platicar, en su telefono aparecio una notificacion de un mensaje con la foto de perfil de mi ex.no supe que decir, y mejor me calle. No se como decirle que es incomodo que hable con mi ex.
emotional assistant: 
Lamento que hayas tenido esa experiencia incómoda con tu amigo y tu ex pareja. A veces, las situaciones pueden ser difíciles de manejar. ¿Quieres hablar más sobre cómo te sientes al respecto?
User: Alex y mi ex, siempre se llevaron bien. No se como decirlo, pero con otros amigos mi ex era mas reservada. Cuando eran fiestas o reuniones no se despegaba de mi, pero si estaba Alex, pues ella podia quedarse platicando horas. Eso antes no me molestaba, pues eran amigos nada mas. pero ahora no se que este pasando entre ellos. Algo me dice que ella siempre estuvo atraida por Alex y el hacia ella.
emotional assistant: 
Lamento que hayas tenido esa experiencia incómoda con tu amigo y tu ex pareja. A veces, las situaciones pueden ser difíciles de manejar. ¿Quieres hablar más sobre cómo te sientes al respecto?
'''

In [ ]:
temperature=.79999999
top_k=70
top_p=.0002333333
num_beams=5
message='''Message here'''
prev=on_message_submit(message,
                       prev,
                       #prev,
                       temperature, 
                       top_k, 
                       top_p, 
                       num_beams)
print(prev)

In [ ]:
len(tokenizer.tokenize(prev))